In [ ]:
import os
os.makedirs("photo_to_3d/inputs", exist_ok=True)
os.makedirs("photo_to_3d/outputs", exist_ok=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving gliad.png to gliad.png


In [ ]:
!unzip photo_to_3d_project.zip

unzip:  cannot find or open photo_to_3d_project.zip, photo_to_3d_project.zip.zip or photo_to_3d_project.zip.ZIP.


In [ ]:
!pip install trimesh plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.0/737.0 kB 10.2 MB/s eta 0:00:00


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!git clone https://github.com/openai/shap-e.git
%cd shap-e
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-dflsgmxl/clip_5a5d3d9e5f1a415bbe6f009c02400ffd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-dflsgmxl/clip_5a5d3d9e5f1a415bbe6f009c02400ffd
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
  Creat

In [ ]:
import torch
from shap_e.models.download import load_model, load_config
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.diffusion.sample import sample_latents
from shap_e.util.notebooks import decode_latent_mesh
import trimesh

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transmitter = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

latents = sample_latents(
    batch_size=1,
    model=model,
    diffusion=diffusion,
    guidance_scale=15.0,
    model_kwargs=dict(texts=["a toy house"]),
    progress=True,
    device=device,

    clip_denoised=True,
    use_fp16=False,
    use_karras=True,
    karras_steps=32,
    sigma_min=0.02,
    sigma_max=200.0,
    s_churn=0.0,
)

decoded_mesh = decode_latent_mesh(transmitter, latents[0])
mesh = decoded_mesh.to_trimesh()

output_path = '/content/photo_to_3d/outputs/model_from_text.obj'
mesh.export(output_path)

/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:09<00:00, 95.8MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
from shap_e.util.notebooks import decode_latent_mesh

torch_mesh = decode_latent_mesh(transmitter, latents[0])

mesh = torch_mesh.tri_mesh()

output_path = '/content/photo_to_3d/outputs/model_from_text.obj'
with open(output_path, 'w') as f:
    mesh.write_obj(f)

print(f"3D model saved to: {output_path}")

In [ ]:
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

mesh = trimesh.load('/content/photo_to_3d/outputs/model_from_text.obj')

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.plot_trisurf(mesh.vertices[:, 0], mesh.vertices[:, 1], mesh.faces,
                mesh.vertices[:, 2], cmap='viridis', linewidth=0.2)

ax.set_title("3D Model Preview")
plt.show()

In [ ]:
from google.colab import files
files.download('/content/photo_to_3d/outputs/model_from_text.obj')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>